In [1]:
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from matplotlib.pyplot import specgram
import keras
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM,BatchNormalization
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Input, Flatten, Dropout, Activation
from keras.layers import Conv1D, MaxPooling1D, AveragePooling1D
from keras.models import Model
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping
from sklearn.metrics import confusion_matrix
import pandas as pd
from tensorflow.keras import optimizers
from keras.callbacks import  History, ReduceLROnPlateau, CSVLogger
from keras import regularizers
import os

In [2]:
import os
import pandas as pd
import librosa
import glob 
import matplotlib.pyplot as plt
import scipy.io.wavfile
import numpy as np
import sys
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
import json

In [3]:
mylist= os.listdir('Audio/')

In [4]:
type(mylist)

list

In [5]:
print(mylist[28])

h8.wav


In [6]:
print(mylist[28][:-2])

h8.w


In [7]:
data, sampling_rate = librosa.load('Audio/h1.wav')

In [8]:
% pylab inline
import os
import pandas as pd
import librosa
import glob 

plt.figure(figsize=(15, 5))
librosa.display.waveplot(data, sr=sampling_rate)

UsageError: Line magic function `%` not found.


In [9]:
feeling_list=[]
for item in mylist:
    if item[:1]=='c':
        feeling_list.append('confident')
    elif item[:1]=='h':
        feeling_list.append('hesitation')

In [10]:
labels = pd.DataFrame(feeling_list)

In [11]:
labels[:30]

,0
0,confident
1,confident
2,confident
3,confident
4,confident
5,confident
6,confident
7,confident
8,confident
9,confident


In [12]:
df = pd.DataFrame(columns=['feature'])
bookmark=0
for index,y in enumerate(mylist):
    if mylist[index][6:-16]!='01' and mylist[index][6:-16]!='07' and mylist[index][6:-16]!='08' and mylist[index][:2]!='su' and mylist[index][:1]!='n' and mylist[index][:1]!='d':
        X, sample_rate = librosa.load('Audio/'+y, res_type='kaiser_fast',duration=20,sr=22050*2,offset=0.5)
        sample_rate = np.array(sample_rate)
        mfccs = np.mean(librosa.feature.mfcc(y=X, 
                                            sr=sample_rate, 
                                            n_mfcc=13),
                        axis=0)
        feature = mfccs
        #[float(i) for i in feature]
        #feature1=feature[:135]
        df.loc[bookmark] = [feature]
        bookmark=bookmark+1

In [13]:
df[:5]

,feature
0,"[-16.820232, -18.45706, -21.608778, -24.4149, ..."
1,"[-7.7447147, -11.703676, -25.93145, -32.009335..."
2,"[-15.132055, -14.900131, -15.525881, -16.52888..."
3,"[-22.422215, -22.12173, -24.500101, -24.51517,..."
4,"[-40.872288, -36.827137, -22.110006, -14.89120..."


In [14]:
df3 = pd.DataFrame(df['feature'].values.tolist())

In [15]:
newdf = pd.concat([df3,labels], axis=1)

In [16]:
rnewdf = newdf.rename(index=str, columns={"0": "label"})

In [17]:
from sklearn.utils import shuffle
rnewdf = shuffle(newdf)
rnewdf[:10]

,0,1,2,3,4,5,6,7,8,9,...,1714,1715,1716,1717,1718,1719,1720,1721,1722,0
28,-40.226589,-41.636909,-42.054363,-41.384113,-40.957066,-41.229168,-41.025276,-40.381077,-40.678017,-39.685406,...,-37.955002,-37.304600,-36.870243,-37.841190,-38.977989,-38.672688,-37.900646,-38.229462,-38.943729,hesitation
1,-7.744715,-11.703676,-25.931450,-32.009335,-37.327023,-40.227337,-38.659184,-37.383896,-37.354702,-33.648838,...,-23.643755,-23.733782,-23.220438,-22.495527,-24.925873,-26.327309,-26.577576,-26.748507,-27.610548,confident
26,-12.421478,-11.878428,-15.390089,-18.352375,-19.291245,-19.885530,-21.758604,-18.974213,-18.627052,-14.877284,...,-16.545683,-18.817238,-19.805643,-18.066191,-14.834322,-12.629364,-10.615643,-9.456968,-5.735039,hesitation
5,-21.062746,-19.874063,-21.405590,-22.539089,-24.753450,-26.102741,-25.752268,-25.848372,-27.458139,-30.682695,...,-30.774059,-30.418831,-30.785896,-25.506216,-14.491749,-10.893973,-8.316031,-9.307844,-10.794649,confident
8,-38.018723,-36.949711,-36.847317,-36.936993,-36.542549,-37.559677,-36.741020,-37.086048,-36.248886,-37.131226,...,-31.100342,-27.289028,-26.005993,-27.416664,-25.266380,-21.899952,-24.512569,-20.669729,-18.057034,confident
15,-23.653240,-20.888134,-19.926292,-21.406864,-21.510973,-20.272482,-22.073172,-24.402040,-22.392355,-21.795790,...,-17.884800,-17.593418,-18.117947,-17.495911,-14.607749,-11.323587,-8.971310,-8.688193,-7.374305,hesitation
29,-23.612978,-24.000324,-30.800642,-32.116688,-34.714756,-35.631748,-36.049080,-36.939442,-30.438988,-21.818970,...,-41.743187,-41.717262,-41.771816,-41.813339,-41.785664,-41.787270,-41.602077,-21.109697,-9.839686,hesitation
18,-22.412477,-24.950457,-27.874882,-26.596357,-27.720085,-30.718983,-31.043692,-32.409920,-33.147598,-33.185978,...,-20.028141,-19.791456,-17.204723,-15.055954,-13.833338,-14.519822,-14.539123,-14.703632,-8.812488,hesitation
27,-11.109602,-12.774125,-17.199146,-18.062941,-21.069395,-22.037626,-22.312897,-22.087980,-22.453606,-25.409731,...,-24.984610,-24.314043,-23.688118,-24.001068,-24.757168,-26.586830,-27.365349,-27.028944,-24.483093,hesitation
19,-9.276253,-10.474096,-12.883884,-13.519767,-15.559297,-16.986559,-16.297722,-15.670814,-17.020214,-15.492110,...,-12.566776,-13.076550,-14.515618,-15.050028,-14.560032,-12.491817,-14.036343,-15.811188,-11.323508,hesitation


In [18]:
rnewdf=rnewdf.fillna(0)

In [19]:
newdf1 = np.random.rand(len(rnewdf)) < 0.8
train = rnewdf[newdf1]
test = rnewdf[~newdf1]

In [20]:
trainfeatures = train.iloc[:, :-1]

In [21]:
trainlabel = train.iloc[:, -1:]

In [22]:
testfeatures = test.iloc[:, :-1]

In [23]:
testlabel = test.iloc[:, -1:]

In [24]:
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder

X_train = np.array(trainfeatures)
y_train = np.array(trainlabel)
X_test = np.array(testfeatures)
y_test = np.array(testlabel)

lb = LabelEncoder()

y_train = np_utils.to_categorical(lb.fit_transform(y_train))
y_test = np_utils.to_categorical(lb.fit_transform(y_test))

C:\Users\user\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [25]:
y_train

array([[0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [26]:
X_train.shape

(26, 1723)

In [27]:
x_traincnn =np.expand_dims(X_train, axis=2)
x_testcnn= np.expand_dims(X_test, axis=2)

In [28]:
model = Sequential()
model.add(Conv1D(256, 8, padding='same',input_shape=(X_train.shape[1],1)))
model.add(Activation('relu'))
model.add(Conv1D(256, 8, padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(MaxPooling1D(pool_size=(8)))
model.add(Conv1D(128, 8, padding='same'))
model.add(Activation('relu'))
model.add(Conv1D(128, 8, padding='same'))
model.add(Activation('relu'))
model.add(Conv1D(128, 8, padding='same'))
model.add(Activation('relu'))
model.add(Conv1D(128, 8, padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(MaxPooling1D(pool_size=(8)))
model.add(Conv1D(64, 8, padding='same'))
model.add(Activation('relu'))
model.add(Conv1D(64, 8, padding='same'))
model.add(Activation('relu'))
model.add(Flatten())
# Edit according to target class no.
model.add(Dense(2))
model.add(Activation('softmax'))
opt = keras.optimizers.SGD(lr=0.0001, momentum=0.0, decay=0.0, nesterov=False)


In [29]:
model.compile(loss='categorical_crossentropy', optimizer=opt,metrics=['accuracy'])

In [30]:
# loading json and creating model
from keras.models import model_from_json
json_file = open('model1.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("saved_models1/Emotion_Voice_Detection_Model.h5")
print("Loaded model from disk")
 
# evaluate loaded model on test data
loaded_model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
score = loaded_model.evaluate(x_testcnn, y_test, verbose=0)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

Loaded model from disk
accuracy: 100.00%


In [31]:
preds = loaded_model.predict(x_testcnn, 
                         batch_size=32, 
                         verbose=1)

1/1 [==============================] - 2s 2s/step


In [32]:
preds

array([[0.88847625, 0.11152377],
       [0.24074464, 0.7592553 ],
       [0.6465491 , 0.3534509 ],
       [0.39057973, 0.6094203 ]], dtype=float32)

In [33]:
preds1=preds.argmax(axis=1)

In [34]:
preds1

array([0, 1, 0, 1], dtype=int64)

In [35]:
abc = preds1.astype(int).flatten()

In [36]:
predictions = (lb.inverse_transform((abc)))

In [37]:
preddf = pd.DataFrame({'predictedvalues': predictions})
preddf[:10]

,predictedvalues
0,confident
1,hesitation
2,confident
3,hesitation


In [38]:
actual=y_test.argmax(axis=1)
abc123 = actual.astype(int).flatten()
actualvalues = (lb.inverse_transform((abc123)))

In [39]:
actualdf = pd.DataFrame({'actualvalues': actualvalues})
actualdf[:10]

,actualvalues
0,confident
1,hesitation
2,confident
3,hesitation


In [40]:
finaldf = actualdf.join(preddf)

In [41]:
finaldf[:5]

,actualvalues,predictedvalues
0,confident,confident
1,hesitation,hesitation
2,confident,confident
3,hesitation,hesitation


In [42]:
finaldf.groupby('actualvalues').count()

,predictedvalues
actualvalues,
confident,2
hesitation,2


In [43]:
finaldf.groupby('predictedvalues').count()

,actualvalues
predictedvalues,
confident,2
hesitation,2


## Testing Path!

In [44]:
% pylab inline
import os
import pandas as pd
import librosa
import glob 

plt.figure(figsize=(15, 5))
librosa.display.waveplot(data, sr=sampling_rate)

UsageError: Line magic function `%` not found.


In [45]:
#livedf= pd.DataFrame(columns=['feature'])
X, sample_rate = librosa.load('profanity1.wav', res_type='kaiser_fast',duration=20,sr=22050*2,offset=0.5)
sample_rate = np.array(sample_rate)
mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=13),axis=0)
featurelive = mfccs
livedf2 = featurelive

In [46]:
livedf2= pd.DataFrame(data=livedf2)

In [47]:
livedf2 = livedf2.stack().to_frame().T

In [48]:
twodim= np.expand_dims(livedf2, axis=2)

In [49]:
livepreds = loaded_model.predict(twodim, 
                         batch_size=32, 
                         verbose=1)

1/1 [==============================] - 0s 211ms/step


In [50]:
livepreds

array([[0.31913793, 0.68086207]], dtype=float32)

In [51]:
livepreds1=livepreds.argmax(axis=1)

In [52]:
liveabc = livepreds1.astype(int).flatten()

In [53]:
livepredictions = (lb.inverse_transform((liveabc)))
print(livepredictions)
print(livepreds[0][0])

['hesitation']
0.31913793
